In [1]:
# Required Imports
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re as re

E:\Program Files (x86)\Anaconda\envs\Projet3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Program Files (x86)\Anaconda\envs\Projet3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
E:\Program Files (x86)\Anaconda\envs\Projet3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Credentials

In [2]:
#Get the Company's Linkedin URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()



Enter the Company Linkedin URL: https://www.linkedin.com/company/dplusforcare/


In [3]:
#accessing Chromedriver
browser = webdriver.Chrome(r'C:\Users\Thibaut\Downloads\chromedriver.exe')

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

C:\Users\Thibaut\AppData\Local\Temp/ipykernel_19560/3556346989.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\Thibaut\Downloads\chromedriver.exe')
C:\Users\Thibaut\AppData\Local\Temp/ipykernel_19560/3556346989.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elementID = browser.find_element_by_id('username')
C:\Users\Thibaut\AppData\Local\Temp/ipykernel_19560/3556346989.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elementID = browser.find_element_by_id('password')


In [12]:
#Go to company post webpage
browser.get(page + 'posts/')


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [37]:
browser.execute_script("document.getElementsByClassName('t-12')[0].click()")

In [39]:
from selenium.webdriver.common.by import By

items = browser.document.getElementsByClassName('t-12')
for item in items:
    text = item.text
    print(text)
    if text == 'idk':
        item.click()
        break

AttributeError: 'WebDriver' object has no attribute 'document'

In [53]:
search_button = browser.find_elements(by=By.CLASS_NAME  , value="social-details-social-counts__comments")

In [55]:
search_button

NameError: name 'li' is not defined

In [32]:
By

selenium.webdriver.common.by.By

In [16]:
#Check out page source code
company_page = browser.page_source  


#Use Beautiful Soup to get access tags
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

In [17]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []


#Looping through the posts and appending them to the lists
for container in containers:
    
    #Try function to make sure its a post and not a promotion
    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper ember-view"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions social-details-social-counts__item"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__comments social-details-social-counts__item"})

        #Appending date and text to lists
        post_dates.append(posted_date.text.strip())
        post_texts.append(text_box.text.strip())


        #Determining media type and collecting relevant info for each type
        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            video_views.append(view_container[1].text.strip().replace(' Views',''))

        except:
            video_views.append('N/A')

        
        #Appending likes and comments if they exist
        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())                           
        except:                                                           
            post_comments.append(0)
            pass
    
    except:
        pass

In [7]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]

In [8]:
data = {
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Post Comments": comment_count,
    "Video Views": video_views,
    "Media Links": media_links
}

df = pd.DataFrame(data)


#Exporting as csv file to program folder
df.to_csv("{}_posts.csv".format(company_name), encoding='utf-8', index=False)

#Export to Excel file to program folder
writer = pd.ExcelWriter("{}_posts.xlsx".format(company_name), engine='xlsxwriter')
df.to_excel(writer, index =False)
writer.save()



In [9]:
containers

[<div class="occludable-update ember-view" id="ember68">
 <div class="feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative feed-shared-update-v2--e2e artdeco-card" data-urn="urn:li:activity:6937770692285222912" id="ember93">
 <div>
 <div role="status">
 <!-- --> </div>
 <h2 class="visually-hidden">
                 Post du fil d’actualité
             </h2>
 <div class="feed-shared-actor display-flex feed-shared-actor--with-control-menu">
 <!-- -->
 <a aria-label="Voir l’entreprise : D+ for care 1 742 abonnés Il y a 18 heures" class="app-aware-link feed-shared-actor__container-link relative display-flex flex-grow-1" data-control-id="PLd5ge+g+mfcereXkq6CCA==" data-control-name="actor_container" href="https://www.linkedin.com/company/dplusforcare/?miniCompanyUrn=urn%3Ali%3Afs_miniCompany%3A40710002" target="_self">
 <div class="feed-shared-actor__container absolute"></div>
 <div class="feed-shared-actor__image relative" data-control-id="PLd5ge+g+mfcereXkq6CCA